In [47]:
from collections import defaultdict
from matplotlib import pyplot as plt
import numpy as np
import operator
import pickle

import src.fusiontools as fusiontools
import src.scoring as scoring
import src.postprocess as postprocess

%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [48]:
path_to_results = 'results/'

In [58]:
# Load best recency model
recency_rank_dic_path = 'ranks_test/recency_beta_7000_rec-all'
with open(recency_rank_dic_path, 'rb') as infile:
    recency_ranks = pickle.load(infile)

    
# Load best knn model
knn_rank_dic_path = 'ranks_test/knn-k-50-rec-all'
with open(knn_rank_dic_path, 'rb') as infile:
    knn_ranks = pickle.load(infile)

# Load best bayesian model
bayes_rank_dic_path = 'ranks_test/bayesian_results_rec_100_rectified'
with open(bayes_rank_dic_path, 'rb') as infile:
    bayes_ranks = pickle.load(infile)

bayes_100 = {}
for mid, recipients in bayes_ranks.items():
    bayes_100[int(mid)] = recipients[:100]

In [59]:
for values in bayes_100.values():
    print(len(values))

100
1
100
14
14
100
100
61
100
100
100
58
100
100
67
100
100
100
100
100
100
82
100
100
100
100
100
77
100
100
81
100
100
100
100
100
100
100
100
100
100
100
100
98
100
100
100
14
100
87
1
100
100
95
76
98
100
95
100
1
100
100
100
100
100
100
100
100
100
100
1
100
100
100
100
14
100
100
77
100
100
99
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
72
100
67
100
100
51
100
51
100
100
100
89
100
100
100
100
100
100
100
100
100
100
100
84
100
65
100
100
84
100
100
67
100
100
100
100
100
100
100
77
100
100
100
100
89
100
100
100
100
36
100
58
100
100
86
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
82
100
100
100
100
100
100
59
100
100
100
100
100
100
1
100
36
100
100
100
100
98
100
100
100
100
100
100
100
100
77
100
100
100
100
100
100
67
77
77
98
57
100
100
100
100
86
100
100
36
36
100
100
100
36
44
98
100
100
100
99
100
100
100
100
100
100
100
51
100
98
100
100
100
100
100
100
51
100
99
100
75
100
14
77
100
100
10

In [60]:
# models = [knn_ranks, recency_ranks]
# weights = [0.20716, 0.39316]

# models = [bayes_100, recency_ranks]
# weights = [0.34218, 0.39316]

models = [bayes_100, recency_ranks, knn_ranks]
weights = [0.34218, 0.39316, 0.20716]


reranking_coeff = 20
fusion_dic = fusiontools.reciprocal_rerank(models, reranking_coeff,  weights=weights)
print(fusion_dic)

            
                

{172034: ['james.d.steffes@enron.com', 'richard.shapiro@enron.com', 'michael.tribolet@enron.com', 'richard.b.sanders@enron.com', 'e..haedicke@enron.com', 'louise.kitchen@enron.com', 'john.lavorato@enron.com', 'steven.j.kean@enron.com', 'rick.buy@enron.com', 'elizabeth.sager@enron.com'], 85335: ['jennifer_lamprecht@oxy.com', 'suzanne.christiansen@enron.com', 'susan.hadix@elpaso.com', 'rhaygood@darbylaw.com', 'mike.grigsby@enron.com', 'pkuykendall@wm.com', 'l..gay@enron.com', 'monique.sanchez@enron.com', 'cathy.morse@foleys.com', 'matthew.lenhart@enron.com'], 40962: ['terry.kowalke@enron.com', 'john.buchanan@enron.com', 'lynn.blair@enron.com', 'raetta.zadow@enron.com', 'ava.garcia@enron.com', 'shelley.corman@enron.com', 'larry.berger@enron.com', 'steven.january@enron.com', 'michael.bodnar@enron.com', 'jean.blair@enron.com'], 118799: ['no.address@enron.com', 'steven.j.kean@enron.com', 'mark.palmer@enron.com', 'karen.denne@enron.com', 'vance.meyer@enron.com', 'meredith.philipp@enron.com', 

In [46]:
postprocess.write_results_ranked(fusion_dic, path_to_results, 'fusion-weighted-reranking-20-bayes-recency-tfidf-knn.csv')